# Concevoir un programme en Python

L’objectif de ce calepin est de mettre en œuvre les bonnes pratiques abordées précédemment en matière de conception d’un programme.

## Rappel sur l’exécution d’un script

Dans le répertoire *scripts* se trouvent plusieurs fichiers Python que l’on peut exécuter avec la commande générique :

```bash
python /path/to/script.py
```

Par exemple :

In [ ]:
! python ./scripts/hello_world.py

Si une erreur apparaît, lisez le message, il vous indique sans doute un problème dans l’expression du chemin vers le fichier.

## Effectuer une régression linéaire

Pour la petite histoire, c’est en 1886 que le terme de régression apparaît pour la première fois, dans un article de Sir Francis Galton qu’il publie sous le titre de *Regression Towards Mediocrity in Hereditary Stature*. Dans cet article, il mettait en évidence que les enfants de personnes de grandes tailles avaient tendance à être plus petits qu’elles, et inversement, d’où l’idée d’une régression vers la médiocrité pour décrire en fait un phénomène d’attraction de la moyenne.

À partir de données simulées, nous calculerons une droite de régression avec la méthode des moindres carrés en appliquant les notions vues jusqu’ici.

**Attention !** Dans le répertoire *scripts*, le fichier *linear-regression.py* nous servira d’ébauche de programme. Toutes les instructions listées devront être reportées dans ce script.

### Charger les modules nécessaires

Chargeons d’abord, à l’endroit prévu à cet effet, toutes les bibliothqèues logicielles qui seront utilisées par notre programme :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### Générer des données aléatoires

La fonction ci-dessous génère des données à l’aspect linéaire. Intégrons-la au programme :

In [ ]:
def generate_data():
    """Generate random data from a linear function with Gaussian noise."""
    noise = np.random.normal(loc=0, scale=10, size=100)
    x = np.arange(100)
    y = 2 * x - 4 + noise

    return x, y

**Astuce :** un doute sur la manière d’utiliser la méthode `np.random.normal()` ? Pensez à [la documentation officielle](https://numpy.org/doc/stable/index.html) ou à la fonction native `help()` :

In [ ]:
help(np.random.normal)

Dans la procédure principale, récupérons les données générées de manière aléatoire et affichons-les dans un diagramme :

In [ ]:
# get data
X, Y = generate_data()

# make plot
coef = np.polyfit(X, Y, 1)
poly1d_fn = np.poly1d(coef)
plt.plot(X, Y, 'yo', X, poly1d_fn(X), '--k')

# save plot
plt.savefig('points.png', dpi=72)

Afin de vérifier le bon fonctionnement du script, exécutons-le depuis une interface en ligne de commande :

In [ ]:
! python ./scripts/linear_regression.py

### Une droite de régression des moindres carrés

Lorsque nous prenons les coordonnées d’une observation au hasard, nous remarquons qu’elles sont éloignées de la droite. Il existe un décalage – que l’on appelle communément une **erreur** – et la droite de régression des moindres carrés est celle qui minimise la somme des carrés de toutes les erreurs.

**Remarque :** Le rapport s’établit au carré afin d’éviter les valeurs négatives.

#### La formule

L’équation réduite qui permet d’obtenir les coordonnées de tous les points de la droite et, partant, d’obtenir une prédiction de $\hat{y}$ en fonction de $x$ respecte la forme $\hat{y} = mx + b$.

Deux étapes majeures pour la trouver :

- calculer d’abord le coefficient directeur $m$
- puis l’ordonnée à l’origine $b$.

#### Calculer le coefficient directeur

La résolution du coefficient directeur d’une droite des moindres carré est régi par la formule ci-dessous :

$$m = \frac{n\sum xy - \sum x \sum y}{n \sum x^2 - \left(\sum x\right)^2}$$

De cette formule, nous concluons avoir besoin de connaître :

- le nombre $n$ des observations ;
- la somme du produit de $x$ et de $y$ ;
- la somme de $x$ et son carré ;
- la somme de $y$ et la somme de $x$ ;
- la somme des carrés de $x$.

La fonction ci-dessous se charge des calculs :

In [ ]:
def slope(x, y):
  """Return the slope of a straight line,
  with the least squares method.

  Arguments:
  x -- data in x-axis
  y -- data in y-axis
  """

  n = len(x)
  sum_xy = sum(x * y)
  sum_x_squared = sum(x) ** 2
  sum_x = sum(x)
  sum_y = sum(y)
  sum_squares_x = sum(x ** 2)

  # formula
  m = (n * sum_xy - sum_x * sum_y) / (n * sum_squares_x - sum_x_squared)

  return m

#### Calculer l’ordonnée à l’origine

La formule de résolution de l’ordonnée à l’origine fait appel au coefficient directeur et aux moyennes des valeurs de $x$ et de $y$ :

$$b = \bar{y} - m\bar{x}$$

Intégrons cette formule dans une fonction :

In [ ]:
def intercept(m, x, y):
  """Intercept of a straight line.
  Arguments:
  m -- slope
  x -- data in x-axis
  y -- data in y-axis
  """

  avg_x = sum(x) / len(x)
  avg_y = sum(y) / len(y)
  b = avg_y - (m * avg_x)

  return b

### Vérification

La droite qui minimise la somme des carrés des erreurs s’établit tout simplement grâce à l’équation réduite de toute droite, à l’exception ici que nous effectuons des prédictions :

$$\hat y = b + mx$$

Ajoutons une fonction qui calcule la coordonnée prédite en *y* d’un point relativement à un $x$ connu :

In [ ]:
def F(*, x, m, b) -> float:
  """Solution to the standard form equation
  of a straight line.

  Keyword-only arguments:
  x -- value of x
  m -- slope
  b -- intercept
  """
  return m * x + b

Dans la procédure principale, ne conservons que la toute première instruction qui génère les données et supprimons tout le reste. À partir des données, faisons appel aux fonction `slope()` et `intercept()` définies précédemment pour récupérer le coefficient directeur et l’ordonnée à l’origine de la droite de régression des moindres carrés :

In [ ]:
# slope & intercept of a straight line
m = slope(X, Y)
b = intercept(m, X, Y)

Ensuite, il ne nous reste plus qu’à effectuer des prédictions pour chaque valeur de $x$ avec la fonction `F()` :

In [ ]:
Y_pred = [ F(x=x, m=m, b=b) for x in X ]

Toujours dans la procédure principale, sauvegardons maintenant le graphique :

In [ ]:
ax = plt.subplots()

ax = plt.plot(X, Y_pred)
ax = plt.scatter(X, Y)

plt.savefig('linear-regression.png', dpi=72)

Exécutons le script. Si un fichier *linear-regression.png* apparaît, vous pouvez maintenant tenter le défi suivant : encapsuler la procédure principale dans une fonction `main()` et, ensuite, rendre le script directement exécutable de telle manière que la commande `./scripts/linear-regression.py` dans un terminal ne renvoie aucune erreur.